# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [4]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [5]:
nutrition_agent = Agent(name="Nutrition Assistant",instructions="""You are a helpful assistant giving out nutrition advice.
                      You give concise answers""")

Let's execute the Agent:

In [6]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy and convenient. Key points:
    
    - Nutrients: good source of potassium (supports blood pressure and heart function), vitamin B6, vitamin C, and dietary fiber.
    - Benefits: convenient, satiating, may aid digestion (especially when not fully ripe due to resistant starch in greener bananas).
    - Things to watch: natural sugars and calories add up if eaten in large amounts; those watching sugar intake or potassium needs may adjust portions.
    - Ripeness note: ripe = sweeter with more sugars; greener = more resistant starch (may be gentler on blood sugar for some people).
    - Special cases: people with kidney disease may need to monitor potassium.
    
    Bottom line: usually a healthy, fiber-rich snack or part of a balanced meal in moderate portions.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for m

Streaming the answer to the screen, token by token

In [7]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are generally healthy as part of a balanced diet.

Key benefits:
- Rich in potassium (good for blood pressure) and vitamin B6
- Vitamin C, dietary fiber (helpful for digestion)
- Resistant starch when unripe (prebiotic benefits)

Things to watch:
- Contain natural sugars; portion size matters if you’re watching carbs or diabetes.
- Peels may carry pesticides if not organic; wash or buy organic if this is a concern (mostly optional).

Quick tips:
- 1 medium banana is a typical serving; 2 is fine for most active people.
- Choose ripe for more flavor and softer texture; greener for more resistant starch.
- Pair with protein or fat (e.g., peanut butter, yogurt) for staying power.

_Good Job!_